In [ ]:
from agno.agent import Agent
from agno.models.ollama import Ollama
from agno.embedder.ollama import OllamaEmbedder
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase
from agno.vectordb.lancedb import LanceDb, SearchType

In [ ]:
OLLAMA_HOST="localhost"
#OLLAMA_HOST="ollama.ollama.svc.cluster.local"

In [ ]:
# Define the database URL where the vector database will be stored
db_url = "/tmp/lancedb"

# Configure the language model
# It's import to increase the context size from the Ollama default of 2048, otherwise the input will be truncated
model = Ollama(id="llama3.1:8b-instruct-q8_0", options={"num_ctx": 4096}, host=OLLAMA_HOST)

# Create Ollama embedder
embedder = OllamaEmbedder(id="nomic-embed-text", dimensions=768)

# Create the vector database
vector_db = LanceDb(
    table_name="recipes",  # Table name in the vector database
    uri=db_url,  # Location to initiate/create the vector database
    embedder=embedder,  # Without using this, it will use OpenAIChat embeddings by default
)


In [ ]:
# Create a knowledge base from a PDF
knowledge_base = PDFUrlKnowledgeBase(
    urls=["https://docs.redhat.com/en-us/documentation/red_hat_openshift_ai_self-managed/2.21/pdf/serving_models/Red_Hat_OpenShift_AI_Self-Managed-2.21-Serving_models-en-US.pdf"],
    vector_db=vector_db
)
knowledge_base.load()

In [ ]:
# Comment out after first run as the knowledge base is loaded
knowledge_base.load()

In [ ]:
agent = Agent(
    model=model,
    markdown=True,
)
agent.print_response("How can LLMs be served on OpenShift AI?", stream=True)

That's ok but not perfect. Can we make it better? Let's include our knowledge base with the RHOAI model serving documentation.

In [ ]:
rag_agent = Agent(
    model=model,
    instructions=["Retrieve relevant OpenShift AI information from the knowledge base"],
    # Add the knowledge base to the agent
    knowledge=knowledge_base,
    show_tool_calls=True,
    markdown=True,
)
rag_agent.print_response("How can LLMs be served on OpenShift AI?", stream=True)

That's a much better answer for OpenShift AI.